<a href="https://colab.research.google.com/github/HubertasVin/deep-learning-practice/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importuojamos reikalingos bibliotekos

In [41]:
!pip install pyngrok

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

# API imports
from flask import Flask, request, jsonify
from pyngrok import ngrok
import requests
from io import BytesIO

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

DATA_DIR = '/content/drive/MyDrive/colab_content'
OI_DATA_DIR = DATA_DIR + "/OpenImages"
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 60
LEARNING_RATE = 1e-6
TRAIN_MODE = True  # Toggle: True to train a new model, False to load saved model
NGROK_AUTH_TOKEN = "2v8l4mW8zOFneFzX7p47qORuEwS_4EcRU64N6iq1mUNEpXN3G"
API_ENABLE = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cpu


Duomenų įkėlimas ir paruošimas

In [42]:
def get_image_paths_and_labels(root_dir):
    classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
    image_paths = []
    labels = []
    for cls in classes:
        images_dir = os.path.join(root_dir, cls, "images")
        if os.path.isdir(images_dir):
            for file in os.listdir(images_dir):
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(images_dir, file))
                    labels.append(class_to_idx[cls])
        else:
            print(f"Warning: {images_dir} not found.")
    return image_paths, labels, classes, class_to_idx

# Get file paths and labels
image_paths, labels, classes, class_to_idx = get_image_paths_and_labels(OI_DATA_DIR)
print("Total images:", len(image_paths))
print("Classes found:", classes)

# Shuffle and split into training (80%) and validation (20%) sets
indices = np.arange(len(image_paths))
np.random.shuffle(indices)
split = int(0.8 * len(image_paths))
train_idx, val_idx = indices[:split], indices[split:]
train_paths = [image_paths[i] for i in train_idx]
train_labels = [labels[i] for i in train_idx]
val_paths   = [image_paths[i] for i in val_idx]
val_labels  = [labels[i] for i in val_idx]

# Define image processing function
def load_and_preprocess_image(path, label, training=True):
    img = tf.io.read_file(path)
    # Use decode_image to support various image formats and force 3 channels
    img = tf.image.decode_image(img, channels=3)
    # Explicitly set the shape to [None, None, 3]
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0  # Scale to [0,1]
    if training:
        img = tf.image.random_flip_left_right(img)
    return img, label


# Create tf.data.Datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(lambda p, l: load_and_preprocess_image(p, l, training=True),
                        num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
val_ds = val_ds.map(lambda p, l: load_and_preprocess_image(p, l, training=False),
                    num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

Total images: 1061
Classes found: ['sandal', 'strawberry', 'traffic light']


Modelio kompiliavimas ir treniravimas

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SIZE + (3,),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(len(classes), activation='softmax')(x)
model = keras.Model(inputs, outputs)
model.summary()

model.compile(optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_path = os.path.join(DATA_DIR, "keras_model.h5")

if TRAIN_MODE:
    history = model.fit(train_ds,
                        epochs=EPOCHS,
                        validation_data=val_ds)
    model.save(model_path)
else:
    model = keras.models.load_model(model_path)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ true_divide_4 (TrueDivide)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ subtract_4 (Subtract)                │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_128 (Functional)    │ (None, 4, 4, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_4           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 3)                   │           3,843 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,261,827 (8.63 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/60
15/27 ━━━━━━━━━━━━━━━━━━━━ 6s 544ms/step - accuracy: 0.2466 - loss: 1.3440

Nupiešti treniravimo ir validacijos nuostolių grafiką

In [ ]:
if TRAIN_MODE:
    plt.figure(figsize=(8,5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

Pagalbinė prognozavimo funkcija

In [ ]:
def predict_image(image):
    # Resize and preprocess
    img = image.resize(IMG_SIZE)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    preds = model.predict(img)
    pred_class = np.argmax(preds, axis=1)[0]
    # Reverse mapping: index -> class name
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    return idx_to_class.get(pred_class, "Unknown")

API skirtas įvesti nuotraukas rankiniu būdu.

Nuotraukos testavimui su API:
- https://img.freepik.com/free-photo/yummy-strawberries-red-mellow-ripe-with-green-leafs-dark-desk_179666-391.jpg
- https://img.freepik.com/free-photo/strawberry-isolated-white-background_1232-1974.jpg
- https://img.freepik.com/free-photo/traffic-light-city-streets_23-2149091964.jpg
- https://img.freepik.com/free-photo/green-traffic-light-intersection_53876-153444.jpg
- https://img.freepik.com/premium-photo/close-up-yellow-street-traffic-light-hanging-from-pole_1048944-23361333.jpg
- https://img.freepik.com/free-photo/summer-slipper-white-shoes-sandals_1203-6528.jpg
- https://img.freepik.com/premium-photo/close-up-shoes-sand_1048944-30578092.jpg
- https://img.freepik.com/free-vector/man-brown-casual-flip-flop-sandal-shoes-vector_53876-25895.jpg

In [ ]:
app = Flask(__name__)

@app.route('/test', methods=['GET'])
def test_endpoint():
    return jsonify({"message": "Test endpoint working!"}), 200

@app.route('/predict', methods=['POST'])
def predict():
    # Allow file upload (multipart/form-data) or JSON with 'url' or 'file_path'
    if "file" in request.files:
        file = request.files["file"]
        try:
            image = tf.keras.preprocessing.image.load_img(file, target_size=IMG_SIZE)
        except Exception as e:
            return jsonify({"error": "Invalid image file."}), 400
    else:
        data = request.get_json(force=True)
        if "url" in data:
            response = requests.get(data["url"])
            image = tf.keras.preprocessing.image.load_img(BytesIO(response.content), target_size=IMG_SIZE)
        elif "file_path" in data:
            image = tf.keras.preprocessing.image.load_img(data["file_path"], target_size=IMG_SIZE)
        else:
            return jsonify({"error": "Provide 'url', 'file_path', or upload a file as 'file'."}), 400

    prediction = predict_image(image)
    return jsonify({"prediction": prediction})

if __name__ == '__main__':
    import sys
    # If run with argument 'api', then start the API server
    if API_ENABLE:
        # Set the ngrok authtoken
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        public_url = ngrok.connect(5000)
        print("Public URL:", public_url)
        app.run(host='0.0.0.0', port=5000)